In [1]:
from MFEA_lib.model import Firework,MFEA1
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import SBX_Crossover
from MFEA_lib.operators.Mutation import Polynomial_Mutation
from MFEA_lib.operators.Selection import ElitismSelection
from MFEA_lib.tasks.Benchmark.Funcs import CEC17_benchmark
from MFEA_lib.tasks.Benchmark.IDPC_EDU import IDPC_EDU_benchmark_1,IDPC_EDU_benchmark_2
from MFEA_lib.EA import *

In [2]:
tasks = IDPC_EDU_benchmark_2.get_tasks(1)
print('num tasks: ', len(tasks))

Reading data...


100%|██████████| 10/10 [00:00<00:00, 54.50it/s]

num tasks:  10


In [3]:
import numpy as np
x = np.random.rand(tasks[0].dim)
tasks[0](x)

99999

In [4]:
class IndClass(Individual):
    def __init__(self, genes, parent= None, dim= None, *args, **kwargs) -> None:
        super().__init__(genes, parent, dim)
        if genes is None:
            self.genes: np.ndarray = np.random.rand(dim)
        self.A = np.ones(len(self.genes))

In [34]:
baseModel = MFEA1.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    crossover= SBX_Crossover(nc = 2),
    mutation= Polynomial_Mutation(nm = 5),
    selection= ElitismSelection()
)
solve = baseModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, rmp = 0.3,
    bound= [0, 1], evaluate_initial_skillFactor= True
)

Time: 02m 40.87s  100 % [====================>]  Cost: 1.30E+01  1.20E+01  1.20E+01  1.60E+01  1.60E+01  1.80E+01  2.20E+01  2.10E+01  2.10E+01  2.70E+01  ,  
END!


In [29]:
gene = baseModel.last_pop[0].__getBestIndividual__.genes
gene

array([0.89335226, 0.44858402, 0.24438358, 0.81417296, 0.6971494 ,
       0.40960454, 0.00571508, 0.95401436, 0.6052526 , 0.00654628,
       0.49849349, 0.59863535, 0.30014745, 0.4617575 , 0.49559978,
       0.58565914, 0.86963799, 0.83222696, 0.77856305, 0.54843311,
       0.61345196, 0.73426973, 0.56636909, 0.77521013, 0.98654561])

In [37]:
gene_list = [baseModel.last_pop[0].ls_inds[i].genes for i in range(len(baseModel.last_pop[0].ls_inds))]
for gene in gene_list:
    cost =0 
    leftDomain = -1
    domain = np.zeros(tasks[0].num_domains+1)
    nodes = np.zeros(tasks[0].num_nodes+1)
    currNode=  tasks[0].source
    for idx in range(len(gene)):
        if currNode == tasks[0].target:
            print(cost)
            break
        l = int(gene[idx]*len(tasks[0].edges[currNode]))
        if l == len(tasks[0].edges[currNode]):
            l = l-1
        # print(len(tasks[0].edges[currNode]))
        # print(tasks[0].edges[currNode])
        nextNode,w,d = tasks[0].edges[currNode][l]
        nodes[nextNode] = 1
        domain[d] = 1
        leftDomain = d
        currNode = nextNode
        cost += w

13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
